In [11]:
!pip install chardet

  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   -------- ------------------------------ 41.0/199.4 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 199.4/199.4 kB 2.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import os
import shutil
from sklearn.model_selection import train_test_split
import json
import cv2
import chardet

In [31]:
#обработка датасета

dir = ('C:/Users/RD/WorkWork/Prog/Datasets/DatasetFenyaTensorflow/combined/')
destination_directoryLabels = ('C:/Users/RD/WorkWork/Prog/Datasets/yolov5/labels_all/')
destination_directoryImages = ('C:/Users/RD/WorkWork/Prog/Datasets/yolov5/images_all/')

#проверка на правильное расширение файла(опционально)
for root, dirs, files in os.walk(dir):
    for file in files:
        if not "txt" in file and not "jpg" in file:
            os.rename(os.path.join(root, file), os.path.join(root, file)+'.jpg')

#сортировка датасета
counter = 0
doppel_counter = 0
new_uid = 0
for root, dirs, files in os.walk(dir):
    for file in files:
        if 'txt' in file:
            for roots1, dirs1, files1 in os.walk(dir):
                for file1 in files1:
                    if 'jpg' in file1 and file[:-3] == file1[:-3]:
                        uid = file1[:-3]
                        
                        #проверка на повторения
                        if uid == new_uid: 
                            print(counter)
                            print(uid)
                            doppel_counter+=1
                        filepath_img = os.path.join(roots1, file1)
                        filepath_txt = os.path.join(root, file)
                        
                        #переформатирование аннотаций 
                        with open(filepath_txt, 'r', encoding='utf-8') as file: 
                            content = file.read()
                            data = json.loads(content)
                            class_name = data['class']
                            height = data['rect']['Height']
                            top_left_x = data['rect']['TopLeftX']
                            top_left_y = data['rect']['TopLeftY']
                            width = data['rect']['Width']
                        image = cv2.imread(filepath_img)
                        img_height, img_width, channels = image.shape
                        x_center = (top_left_x + width / 2) / img_width
                        y_center = (top_left_y + height / 2) / img_height
                        heightN = height / img_height
                        widthN = width / img_width
                        if class_name == 'sand':
                            temp_clas = 2
        
                        if class_name == 'gravel':
                            temp_clas = 1
        
                        if class_name == 'empty':
                            temp_clas = 0
                        new_data = f"{temp_clas} {x_center} {y_center} {widthN} {heightN}"
                        with open(filepath_txt, 'w', encoding='utf-8') as file:
                            file.write(new_data)
                    #копирование файлов
                    new_file = shutil.copy(os.path.join(root, file), destination_directoryLabels + 'instance' + str(counter) +'.txt')
                    new_file1 = shutil.copy(os.path.join(root, file1), destination_directoryImages + 'instance' + str(counter) +'.jpg')
                    counter+=1
                    new_uid = file1[:-3]
print('повторений:' + str(doppel_counter))
print('всего:' + str(counter))











FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/RD/WorkWork/Prog/Datasets/DatasetFenyaTensorflow/combined/Dataset\\Dataset\\sorted\\EMPTY100.jpg'

In [ ]:
#рандомное разбиение датасема

dataset_dir = 'C:/Users/RD/WorkWork/Prog/Datasets/yolov5'
images_dir = os.path.join(dataset_dir, 'images_all')
labels_dir = os.path.join(dataset_dir, 'labels_all')
output_dirs = {
    'train': os.path.join(dataset_dir, 'train'),
    'val': os.path.join(dataset_dir, 'val'),
    'test': os.path.join(dataset_dir, 'test')
}
split_ratios = [0.8, 0.1, 0.1]
random_seed = 42


for key, output_dir in output_dirs.items():
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)


image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.txt')])


assert len(image_files) == len(label_files), "Количество изображений и меток должно совпадать"

# Разделяем на train и временный (val + test)
train_images, temp_images, train_labels, temp_labels = train_test_split(
    image_files, label_files, test_size=(split_ratios[1] + split_ratios[2]), random_state=random_seed)

# Разделяем временный на val и test
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=split_ratios[2] / (split_ratios[1] + split_ratios[2]), random_state=random_seed)


def copy_files(file_list, src_dir, dst_dir):
    for file_name in file_list:
        shutil.copy(os.path.join(src_dir, file_name), os.path.join(dst_dir, file_name))


copy_files(train_images, images_dir, os.path.join(output_dirs['train'], 'images'))
copy_files(train_labels, labels_dir, os.path.join(output_dirs['train'], 'labels'))

copy_files(val_images, images_dir, os.path.join(output_dirs['val'], 'images'))
copy_files(val_labels, labels_dir, os.path.join(output_dirs['val'], 'labels'))

copy_files(test_images, images_dir, os.path.join(output_dirs['test'], 'images'))
copy_files(test_labels, labels_dir, os.path.join(output_dirs['test'], 'labels'))

print("Датасет успешно разбит на train, val и test с использованием seed!")